# Arrays

# Load Packages

In [1]:
using Compat, Missings        #in Julia 0.6 
#using Dates, DelimitedFiles  #in Julia 0.7   

include("printmat.jl")   #a function for prettier matrix printing

printlnPs (generic function with 2 methods)

# Scalars, Vectors and Multi-dimensional Arrays

*are different things*, even if they happen to "look" similar. For instance, a 1x1 array is not a scalar and a nx1 array is not a vector. This is discussed in some detail further down. 

However, we first present some common features of all arrays (vectors or multi-dimensional arrays).

# Creating Arrays

The typical ways of getting an array are 

* hardcoding the contents
* reading in data from a file
* as a result from computations
* allocating the array and then changing the elements
* (often not so smart) growing the array by adding rows (or colums,..)
* by list comprehension

The next few cells give simple examples.

## 1. Hardcoding the Contents or Reading from a File

In [2]:
z = [11 12;                       #typing in your matrix
     21 22]                       #line breaks mean new rows
println("A matrix that we typed in:")
printmat(z)

x = readdlm("Data/MyData.csv",',',skipstart=1)  #read matrix from file    
println("First four lines of x from csv file:") 
printmat(x[1:4,:])

A matrix that we typed in:
        11        12
        21        22

First four lines of x from csv file:
197901.000     4.180     0.770    10.960
197902.000    -3.410     0.730    -2.090
197903.000     5.750     0.810    11.710
197904.000     0.050     0.800     3.270



## 2a. Allocating an Array and Then Changing the Elements: Fill

An easy way to create an array is to use the `fill()` function.

```
A = fill(0,(10,2))           #10x2, integers (0)
B = fill(0.0,10)             #vector with 10 elements, floats (0.0)
C = fill(NaN,(10,2))         #10x2, floats (NaN)
D = fill("",3)               #vector with 3 elements, strings ("")
E = fill(Date(1),3)          #vector with 3 elements, dates (0001-01-01) 
```

In [3]:
x = fill(0.0,(3,2))     #allocates a 3x2 matrix, (to be filled with floats)
println("so far, x is filled with 0.0. For instance, x[1,1] is $(x[1,1])")

for i = 1:size(x,1), j = 1:size(x,2)
    x[i,j] = i/j
end

println("\nx after some computations")
printmat(x)

so far, x is filled with 0.0. For instance, x[1,1] is 0.0

x after some computations
     1.000     0.500
     2.000     1.000
     3.000     1.500



## 2b. Allocating an Array and Then Changing the Elements: A More General Approach (extra)

You can also create an array by 

```
A = Array{Int}(undef,10,2)                #10x2, integers
B = Array{Float64}(undef,10)              #vector with 10 elements, floats
```

The ```undef``` is needed (and signals that the matrix is yet not initialized).

This is a bit faster than `fill()`, and also more flexible since you can declare things like
```
F = Array{Any}(undef,3)                #vector with 3 elements, can include anything
G = Array{Array{Int}}(undef,4)         #vector of vectors (of integers)
```

An `Any` array can have elements of different types.

In [4]:
F = Compat.Array{Any}(undef,3)    #0.7 syntax
F[1] = [1;2;3;4]             #F[1] contains a vector
F[2] = "Sultans of Swing"    #F[2] a string
F[3] = 1978                  #F[3] an integer 

printmat(F)

[1, 2, 3, 4]
Sultans of Swing
      1978



## 3. Growing an Array

Growing a matrix is slow, so try to avoid it. Instead, pre-allocate the matrix and then fill it (see above). In case you really have to grow a matrix, use `vcat()` and/or `hcat()`.

However, growing a *vector* is not that slow. It can be done by 
```
push!(old vector,new element1,new element 2)
```

If you instead want to add all elements of a vector, then do
```
append!(old vector,new vector)
```

In [5]:
B = Float64[]                 #empty vector, to include floats
for i = 1:3
    x_i = 2.0 + 10^i
    push!(B,x_i)              #adding an element at the end
end 
println("a vector with 3 elements:")
printmat(B)

a vector with 3 elements:
    12.000
   102.000
  1002.000



## 4. List Comprehension and map (extra)

List comprehension sounds fancy, but it is just a simple way to create arrays from repeated calculations. Similar to a "for loop."

You can achieve the same thing with map.

In [6]:
A = [collect(1:i) for i=1:3]         #this creates a vector of vectors

println("A[1] is vector with 1 element, A[2] a vector with 2 elements,...")
printmat(A)

A = map(i->collect(1:i),1:3)         #map(f,x) applies the function f to 
printmat(A)                          #each element of x

A[1] is vector with 1 element, A[2] a vector with 2 elements,...
       [1]
    [1, 2]
 [1, 2, 3]

       [1]
    [1, 2]
 [1, 2, 3]



# (Elementwise) Functions of Arrays

Let `X` be an array, and `a` and `b` be scalars. Then, `y = fn.(X,a,b)` will generate an array `y` where `y[i,j] = fn(X[i,j],a,b)`

(You could achieve the same thing with ```map(xi->fn(xi,a,b),x)```.)

In [7]:
fn(x,a,b) = a/x + b             #x has to be a scalar for this to work

fn (generic function with 1 method)

In [8]:
X = [11 12;
     101 102;
     1001 1002]

printmat(fn.(X,100,10))

    19.091    18.333
    10.990    10.980
    10.100    10.100



# Basic Linear Algebra

The next few cells demonstrate some simple linear algebra.

In [9]:
A = [11 12;
     21 22]
B = [1 2; 
     0 10]

println("A and B")
printmat(A)
printmat(B)

A and B
        11        12
        21        22

         1         2
         0        10



## Transposing

In [10]:
A = [11 12;
     21 22]

println("A:")
printmat(A)

println("\n","the transpose A': ")  
printmat(A')

A:
        11        12
        21        22


the transpose A': 
        11        21
        12        22



## Adding and Multiplying Matrices

In [11]:
B = [1 2; 
     0 10]

println("\n","A + B")             #matrix addition
printmat(A+B)

println("\n","A.*B")              #element-by-element multiplication
printmat(A.*B)


println("\n","A*B")              #matrix multiplication
printmat(A*B)                    


A + B
        12        14
        21        32


A.*B
        11        24
         0       220


A*B
        11       142
        21       262



## Matrix Inverse

In [12]:
println("\n","inv(A)") 
printmat(inv(A))

println("\n","inv(A)*A") 
printmat(inv(A)*A)


inv(A)
    -2.200     1.200
     2.100    -1.100


inv(A)*A
     1.000     0.000
     0.000     1.000



## Matrix .+ Scalar, Matrix * Scalar

To add or subtract a scalar from each element in an array, use `.+` and `.-` as in
```
A .+ 2 or A .- 2
```
Notice the dot. (Watch out when the number comes first: ```2.+A``` is not allowed since it is ambigious. However, ```2.0.+A``` and ```2 .+ A``` both work.)

To multiply or divide, just do 
```
A * 2 or A/2
```
(no dot is needed)

In [13]:
println("A .+ 2")
printmat(A .+ 2)

println("A * 2")
printmat(A*2)

A .+ 2
        13        14
        23        24

A * 2
        22        24
        42        44



# Reshuffling Matrices

The cells below show how to pick out specific rows and columns from a matrix, how to stack matrices, and some other types of reshuffling. 

## Using Parts of a Matrix

The most common way to use parts of an array is by indexing. For instance, to use the second column of A, do
```
A[:,2]
```

Notice that 
```
A[1,:]
```
gives a (column) vector (yes, it does), while
```
A[1:1,:]
```
gives a 1xk matrix. (It looks like a row vector, but is really a matrix.)

In [14]:
A = [11 12;
    21 22]
println("A:")
printmat(A)

println("\nsecond column of A:")
printmat(A[:,2])

println("\n","first row of A (as a vector): ")
printmat(A[1,:])                          #notice 1 makes it a vector


println("\n","first row of A: ")
printmat(A[1:1,:])                        #use 1:1 to keep it as a 1x2 matrix

A:
        11        12
        21        22


second column of A:
        12
        22


first row of A (as a vector): 
        11
        12


first row of A: 
        11        12



### Stacking Matrices (Concatenating)

In [15]:
z = [A;B]                #same as vcat(A,B)                    
println("\n","stacking A and B vertically")  
printmat(z)

z2 = [A B]                 #same as hcat(A,B)
println("\n","stacking A and B horizontally")  
printmat(z2) 


stacking A and B vertically
        11        12
        21        22
         1         2
         0        10


stacking A and B horizontally
        11        12         1         2
        21        22         0        10



## Splitting up an Array

Sometimes you want to assign separate names to the columns (or rows) of a matrix. The next cell shows an example.

In [16]:
println("A simple way...which works well when you want to create a few variables")
x1 = A[:,1]                    
x2 = A[:,2]
printmat(x2)

println("Another, prettier way")
(z1,z2) = [A[:,i] for i = 1:2]    
printmat(z2)

A simple way...which works well when you want to create a few variables
        12
        22

Another, prettier way
        12
        22



## A View into an Array

Creating new variables (as in "Splitting up") can be slow and a waste of memory. Instead, you may create a *view* into the array and give it a name. 

Note that this is not an independent objexct: it just creates a reference to that part of the original array. In particular, if you change an element of this view, then you also change the original array, and vice versa. (Try changing an element of `y2` below and see what it does to `A`.)

In [17]:
y2 = view(A,:,2)             #column 2 of A

println("A[:,2] but with a separate name (y2):")
printmat(y2)

A[:,2] but with a separate name (y2):
        12
        22



# Arrays vs. Vectors vs. Scalars

Matrices, vectors and scalars are different things, even if they contain the same number of elements. In particular,

(a) an $n\times1$ matrix is not the same thing as an $n$ vector

(b) a $1\times1$ matrix or a 1-element vector are not the same thing as a scalar.

As you will see further on, vectors are often more convenient than $n\times1$ matrices.

In [18]:
A = ones(3,1)         #this is a 3x1 matrix
B = ones(3)           #a vector with 3 elements

println("The sizes of matrix A and vector B: $(size(A)) $(size(B))")

println("\nTesting if A==B: ",isequal(A,B))

println("\nThe nx1 matrix A and n-element vector B can often be used together, for instance, as in A+B, whose size is ",size(A+B))
printmat(A+B)

The sizes of matrix A and vector B: (3, 1) (3,)

Testing if A==B: false

The nx1 matrix A and n-element vector B can often be used together, for instance, as in A+B, whose size is (3, 1)
     2.000
     2.000
     2.000



To convert a 1-element vector or 1x1 matrix `C` to a scalar, just extract its first element
```
myScalar = C[1]
```

To test if `C` is a 1-element vector or matrix and then automatically convert it to a scalar, do 

```
if length(C) == 1 && !isa(C,Number)
    C = C[1]
end    
```

In [19]:
C = ones(1,1)                   #a 1x1 matrix
c = 1                           #a scalar

println("\nc/C would give an error since C is a (1x1) matrix")

println("\nInstead, do c/C[1]: ",c/C[1])


if length(C) == 1 && !isa(C,Number)
    C = C[1]
end    
println("\nAfter conversion of C, do c/C: ",c/C)


c/C would give an error since C is a (1x1) matrix

Instead, do c/C[1]: 1.0

After conversion of C, do c/C: 1.0


# Vectors: x'x and x'A*x Create Scalars (if x is a vector)

If $x$ is a vector and $A$ a matrix, then $x'x$ and $x'A*x$ are scalars. This is what a linear algebra text book would teach you, so vectors are very useful.

This is *not* true if x is an array of size $n\times1$. In that case the result is a 1x1 array. 

Recommendation: use vectors (instead if $nx1$ arrays) when you can.

In [20]:
x = [1;2]                                 #this is a vector
A = [11 12;
     21 22]
println("\nx'x and x'A*x when x is a 2 element vector: ",x'x," ",x'A*x)


x   = zeros(2,1)                   #this is a 2x1 matrix (array) 
x[1] = 1
x[2] = 2
println("\nx'x and x'A*x when x is a 2x1 array: ",x'x," ",x'A*x)


x'x and x'A*x when x is a 2 element vector: 5 165

x'x and x'A*x when x is a 2x1 array: [5.0] [165.0]


# An Array of Arrays (and other things)

If `x1` and `x2` are two arrays, then `y=[x1,x2]` is a vector (of arrays) where `y[1] = x1` and `y[2] = x2`.

You do the same even if the elements are strings or other things, for instance, when `x1 = "A nice dog"`.

In [21]:
x1 = ones(3,2)
x2 = [1;2]
y = [x1,x2]               #a vector of arrays
println(size(y))
printmat(y[1])
printmat(y[2])

x1 = "A nice dog"
x2 = [1;2]
y = [x1,x2]              #a vector of string and arrays
println(size(y))
println(y[1])
printmat(y[2])

(2,)
     1.000     1.000
     1.000     1.000
     1.000     1.000

         1
         2

(2,)
A nice dog
         1
         2



# Arrays are Different...

Vectors and matrices (arrays) can take lots of memory space, so **Julia is designed to avoid unnecessary copies of arrays**. In short, notice the following

* ```B = A``` creates two names of the *same* array (changing one changes the other)
* ```B = reshape(A,n,m)```, ```B = vec(A)```, and (in 0.7) ```B = A'``` and create *another view* of the same array (changing one changes the other)
* When an you input an array to a function, then this array is shared between the function and the calling program (scope). Changing *elements* of the array (inside the function) will then change the array outside the function. The next cell provides some details.

In [22]:
function f1(A)
    A[1] = A[1]/2          #changes ELEMENTS of A, affects outside value
  return A
end

x  = [1.0 2.0]
printlnPs("original x:            ",x)

y1 = f1(x)
printlnPs("x after calling f1(x): ",x)

original x:                 1.000     2.000
x after calling f1(x):      0.500     2.000


You can circumvent this behaviour by making an explicit copy as in 
```
function f3(A)
    B    = copy(A)                
    B[1] = B[1]/2          
  return B
end
```

Also, notice that this "problem" does not occur when you change or redefine the whole matrix (rather than just some individual elements) as in 
```
function f2(A)
    A = A/2                #changing the entire matrix, or
    A = ones(3,1)          #creating a new matrix 
  return A
end
```